In [ ]:
import numpy as np
import pandas as pd
import pathlib
from tqdm.auto import tqdm

import ray

In [ ]:
import os
os.path.abspath("Data/StudyData/BiHealth_20230313.dta")

In [ ]:
base_path = "/home/jakobs"

In [ ]:
endpoints_md = pd.read_csv(f"{base_path}/BiHealth/onnx/endpoints.csv").drop(columns="Unnamed: 0").set_index("endpoint")#[["endpoint", "eligable", "n", "freq", "phecode", "phecode_string", "phecode_category", "sex"]]
endpoints_md.reset_index().to_feather(f"{base_path}/data/endpoints_md.feather")

In [ ]:
data_baseline = pd.read_feather(f"{base_path}/data/data_baseline_230321.feather")[["eid"]]

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# create a sample DataFrame with unique eids

# set the random seed for reproducibility
random_state = 42

# create empty lists to store the train and test sets
train_dfs = []
test_dfs = []

n_splits = 10
data_baseline["partition"] = np.random.randint(0, n_splits, size=len(data_baseline))

for i in range(n_splits):
    # create a set of eids for the test set that are mutually exclusive with the previous test sets
    test_eids = data_baseline.query("partition==@i").eid
    
    # split the DataFrame into train and test sets based on the selected eids
    test_df = data_baseline[data_baseline["eid"].isin(test_eids)].assign(split="test")
    train_df = data_baseline[~data_baseline["eid"].isin(test_eids)].assign(split="train").assign(partition=i)

    # append the train and test sets to the lists
    train_dfs.append(train_df)
    test_dfs.append(test_df)

# concatenate the train and test sets into a new long DataFrame
train_test_df = pd.concat(train_dfs + test_dfs, ignore_index=True)[["partition", "split", "eid"]].sort_values(["partition", "split", "eid"]).reset_index(drop=True)

# display the result
train_test_df

In [ ]:
train_test_df.query("split=='train'").value_counts("partition")

In [ ]:
train_test_df.to_feather(f"{base_path}/data/splits_230321.feather")